In [20]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
%matplotlib inline

# File paths
file_path = '../Emission factor tool/Raw eGRID EF.xlsx'
gwp_file_path = '../Emission factor tool/GWP.xlsx'

# Load the data
df = pd.read_excel(file_path, engine='openpyxl')
gwp_df = pd.read_excel(gwp_file_path)

# Display the first few rows to ensure the data loaded correctly
print(df.head())
print(gwp_df.head())

# Extract relevant GWP values from the AR5 column
gwp_values = {
    'CO2': gwp_df[gwp_df['Global Warming Potential'] == 'CO2']['AR5'].values[0],
    'CH4': gwp_df[gwp_df['Global Warming Potential'] == 'CH4']['AR5'].values[0],
    'N2O': gwp_df[gwp_df['Global Warming Potential'] == 'N2O']['AR5'].values[0]
}
print(gwp_values)

# Conversion factor from lb/MWh to mt/kWh (corrected)
conversion_factor = 4.53592e-7

# Function to convert raw factors to mtCO2e/kWh
def convert_to_mtco2e(co2, ch4, n2o):
    co2_mt = co2 * conversion_factor * gwp_values['CO2']
    ch4_mt = ch4 * conversion_factor * gwp_values['CH4']
    n2o_mt = n2o * conversion_factor * gwp_values['N2O']
    total_mtco2e = co2_mt + ch4_mt + n2o_mt
    return total_mtco2e

# Function to get emission factors, convert them, and display GWP values
def get_emission_factors_and_convert(acronym):
    result = df[df['eGRID Subregion Acronym'].str.upper() == acronym.upper()]
    if not result.empty:
        co2 = result['CO2 Factor (lb / MWh)'].values[0]
        ch4 = result['CH4 Factor (lb / MWh)'].values[0]
        n2o = result['N2O Factor (lb / MWh)'].values[0]
        total_mtco2e = convert_to_mtco2e(co2, ch4, n2o)
        
        # Print GWP values
        print(f"GWP Values Used: CO2 = {gwp_values['CO2']}, CH4 = {gwp_values['CH4']}, N2O = {gwp_values['N2O']}")
        
        return {
            'CO2 (mt/kWh)': co2 * conversion_factor * gwp_values['CO2'],
            'CH4 (mt/kWh)': ch4 * conversion_factor * gwp_values['CH4'],
            'N2O (mt/kWh)': n2o * conversion_factor * gwp_values['N2O'],
            'Total CO2e (mtCO2e/kWh)': total_mtco2e
        }
    else:
        return "Acronym not found!"

# Example of providing the acronym input directly
acronym_input = "SRSO"

# Get the factors and converted result for the provided acronym
factors_converted = get_emission_factors_and_convert(acronym_input)

# Display the result
factors_converted

  eGRID Subregion Acronym       eGRID Subregion Name  CO2 Factor (lb / MWh)  \
0                    AKGD    AKGD (ASCC Alaska Grid)                 1067.7   
1                    AKMS  AKMS (ASCC Miscellaneous)                  485.2   
2                    AZNM      AZNM (WECC Southwest)                  819.7   
3                    CAMX     CAMX (WECC California)                  531.7   
4                    ERCT           ERCT (ERCOT All)                  813.6   

   CH4 Factor (lb / MWh)  N2O Factor (lb / MWh)  
0                  0.091                  0.012  
1                  0.025                  0.004  
2                  0.052                  0.007  
3                  0.031                  0.004  
4                  0.054                  0.008  
  Global Warming Potential     SAR    AR4    AR5
0                      CO2     1.0      1      1
1                      CH4    21.0     25     28
2                      N2O   310.0    298    265
3                   CFC-11  3

{'CO2 (mt/kWh)': 0.00040455870479999995,
 'CH4 (mt/kWh)': 8.509385920000001e-07,
 'N2O (mt/kWh)': 1.2020188000000001e-06,
 'Total CO2e (mtCO2e/kWh)': 0.00040661166219199994}